In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

print("API Key is set:", bool(os.environ.get('OPENAI_API_KEY')))

API Key is set: True


In [2]:
from agents import Agent, function_tool, Runner
from pydantic import BaseModel
from typing import List, Optional
import json

class BenchPoint(BaseModel):
    percentile: str   # "p25", "median", "p75", "p90"
    total_comp: int   # USD, all-in (base + equity + bonus)

@function_tool
def search_salary_data(company: str, role: str, location: Optional[str] = None) -> str:
    """
    Search for salary data using OpenAI's search capabilities.
    """
    from openai import OpenAI
    client = OpenAI()
    
    search_query = f"current salary ranges and compensation data for {role} position at {company}"
    if location:
        search_query += f" in {location}"
    search_query += " from levels.fyi, glassdoor, or other reliable sources"
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{
            "role": "user",
            "content": f"Search for and summarize: {search_query}"
        }]
    )
    return response.choices[0].message.content

@function_tool
def basic_stats(nums: List[int]) -> dict:
    """Calculate basic statistics for a list of numbers."""
    if not nums:
        return {"mean": 0, "stdev": 0, "min": 0, "max": 0}
    return {
        "mean": int(stats.mean(nums)),
        "stdev": int(stats.stdev(nums)) if len(nums) > 1 else 0,
        "min": min(nums),
        "max": max(nums)
    }

CompBenchAgent = Agent(
    name="CompBenchAgent",
    instructions="""
You receive {"company": …, "role": …, "location": … }.
1. Use search_salary_data to find salary information.
2. Parse the search results to extract salary ranges and percentiles.
3. Return a JSON object in this EXACT format:
   {
     "benchmarks": [
       {"percentile": "p25", "total_comp": number},
       {"percentile": "median", "total_comp": number},
       {"percentile": "p75", "total_comp": number},
       {"percentile": "p90", "total_comp": number}
     ],
     "summary": "One-sentence human summary"
   }
IMPORTANT:
- Return ONLY the JSON object, no other text
- Use "p25", "median", "p75", "p90" as percentile values
- Use "total_comp" as the key for compensation
- Make sure all numbers are integers
- Do not include any markdown formatting
""",
    tools=[search_salary_data, basic_stats],
)

In [3]:
SanityCheckAgent = Agent(
    name="SanityCheckAgent",
    instructions="""
Input:
  { "claimed_comp": 250000, "benchmarks": [...] }

Compare claimed_comp to the percentiles and return a JSON object in this EXACT format:
{
  "verdict": "low" | "plausible" | "high" | "outlier",
  "rationale": "Clear explanation of why this verdict was chosen"
}

IMPORTANT:
- Return ONLY the JSON object, no other text
- Use exactly one of: "low", "plausible", "high", "outlier" for verdict
- Do not include any markdown formatting
""",
    tools=[basic_stats],
)

In [4]:
NegotiatorAgent = Agent(
    name="NegotiatorAgent",
    instructions="""
You are a salary-negotiation coach embedded in the hiring workflow.

Input bundle:
{
  "candidate":        {...},
  "bench_summary":    "<string from CompBenchAgent>",
  "sanity":           { "verdict": ..., "rationale": ... },
  "manager_limits":   { "max_total_comp": ..., "pref_total_comp": ... }
}

Return a JSON object in this EXACT format:
{
  "proposed_offer": {
    "base": number,
    "equity": number,
    "bonus": number,
    "signon": number
  },
  "negotiation_script": "Step-by-step script for the negotiation",
  "risk_notes": "Key points to consider and potential risks"
}

IMPORTANT:
- Return ONLY the JSON object, no other text
- All numbers must be integers
- Total compensation must be within manager_limits
- Do not hallucinate any numbers and that the numbers make sense and are reasonable and are within the manager_limits.
- Do not include any markdown formatting
""",
    tools=[basic_stats],
)

In [5]:
async def run_negotiation(flow_input: dict):
    """
    flow_input needs:
      company, role, location, claimed_comp, manager_limits, candidate
    """
    def clean_json_response(response):
        """Strip markdown formatting and extract JSON from response"""
        text = response.final_output
        # Find the JSON block
        if "```json" in text:
            # Extract everything between ```json and ```
            json_text = text.split("```json")[1].split("```")[0]
            return json_text.strip()
        return text.strip()
    
    bench_response = await Runner.run(CompBenchAgent, [{
        "role": "user",
        "content": json.dumps({
            "company":  flow_input["company"],
            "role":     flow_input["role"],
            "location": flow_input.get("location")
        })
    }])
    
    print("Bench response:", bench_response.final_output)
    
    try:
        bench = json.loads(clean_json_response(bench_response))
    except json.JSONDecodeError as e:
        print(f"Error decoding benchmark response: {e}")
        print("Raw response:", bench_response.final_output)
        return None
    
    sanity_response = await Runner.run(SanityCheckAgent, [{
        "role": "user",
        "content": json.dumps({
            "claimed_comp": flow_input["claimed_comp"],
            "benchmarks":   bench["benchmarks"]
        })
    }])
    
    print("Sanity response:", sanity_response.final_output)
    
    try:
        sanity = json.loads(clean_json_response(sanity_response))
    except json.JSONDecodeError as e:
        print(f"Error decoding sanity response: {e}")
        print("Raw response:", sanity_response.final_output)
        return None
    
    outcome_response = await Runner.run(NegotiatorAgent, [{
        "role": "user",
        "content": json.dumps({
            **flow_input,
            "bench_summary": bench["summary"],
            "sanity":        sanity
        })
    }])
    
    print("Outcome response:", outcome_response.final_output)
    
    try:
        outcome = json.loads(clean_json_response(outcome_response))
    except json.JSONDecodeError as e:
        print(f"Error decoding outcome response: {e}")
        print("Raw response:", outcome_response.final_output)
        return None
    
    return outcome

# Example usage
example = {
    "company":        "Google",
    "role":           "Software Engineer",
    "location":       "San Francisco",
    "claimed_comp":   245_000,
    "manager_limits": {
        "max_total_comp": 260_000,
        "pref_total_comp": 230_000,
        "must_offer_signon": True
    },
    "candidate": {"name": "Jane Doe"}
}

# Run the async function
result = await run_negotiation(example)
print("\nFinal result:", result)

Bench response: {
  "benchmarks": [
    {"percentile": "p25", "total_comp": 188000},
    {"percentile": "median", "total_comp": 267000},
    {"percentile": "p75", "total_comp": 392000},
    {"percentile": "p90", "total_comp": 552000}
  ],
  "summary": "Software Engineers at Google in San Francisco have total compensation ranging from $188,000 at the 25th percentile to $552,000 at the 90th percentile, depending on seniority level."
}
Sanity response: {
  "verdict": "plausible",
  "rationale": "The claimed compensation of 245000 falls between the 25th percentile (188000) and the median (267000), making it a plausible value within the provided benchmarks."
}
Outcome response: {
  "proposed_offer": {
    "base": 150000,
    "equity": 50000,
    "bonus": 20000,
    "signon": 10000
  },
  "negotiation_script": "1. Present the base salary of $150,000, explaining that it is competitive and aligns well with the current market benchmarks for similar roles in San Francisco. 2. Highlight the equit